In [60]:
import reverb
import tensorflow as tf

In [65]:
table=reverb.Table(
                 name='table',
                 sampler=reverb.selectors.Uniform(),
                 remover=reverb.selectors.Fifo(),
                 max_size=65000,
                signature=
            [
                tf.TensorSpec([None,None,2],
                              tf.float32),
           
                tf.TensorSpec([],
                              tf.float32)]
        ,
    
                 rate_limiter=reverb.rate_limiters.MinSize(1)
            )
reverb_server = reverb.Server(tables=[table], port=8000)

[reverb/cc/platform/tfrecord_checkpointer.cc:162]  Initializing TFRecordCheckpointer in /tmp/tmpwfb2rtdk.
[reverb/cc/platform/tfrecord_checkpointer.cc:567] Loading latest checkpoint from /tmp/tmpwfb2rtdk
[reverb/cc/platform/default/server.cc:71] Started replay server on port 8000
[reverb/cc/platform/default/server.cc:84] Shutting down replay server


In [66]:
client=reverb_server.localhost_client()

In [67]:
import mpg.ml.dataset.generator as gen
import numpy as np

In [68]:
with client.writer(max_sequence_length=1) as writer:
    for n in  [1,2,3,5]:
        A=tf.random.uniform(shape=[n,n])
        W=tf.random.uniform(shape=[n,n])
        v=tf.random.uniform(shape=())
        writer.append([tf.stack([A,W],axis=-1),v])
        writer.create_item(table="table",priority=1,num_timesteps=1)
    writer.flush()

In [70]:
client.server_info()["table"].current_size

4

In [58]:
dataset=client.sample("table",1,emit_timesteps=False)

In [59]:
for k in dataset:
    print(k.data[0].shape)
    break

(2, 2, 2, 2)


[reverb/cc/client.cc:165] Sampler and server are owned by the same process (2530229) so Table table is accessed directly without gRPC.


In [4]:
with client.trajectory_writer(num_keep_alive_refs=3) as writer:
    writer.append({'a': 2, 'b': 12})
    writer.append({'a': 3, 'b': 13})
    writer.append({'a': 4, 'b': 14})

    # Create an item referencing all the data.
    writer.create_item(
      table='table',
      priority=1.0,
      trajectory={
          'a': writer.history['a'][:],
          'b': writer.history['b'][:],
      })

    # Block until the item has been inserted and confirmed by the server.
    writer.flush()

In [ ]:
print(list(client.sample('table', num_samples=2)))


[reverb/cc/client.cc:165] Sampler and server are owned by the same process (1493666) so Table table is accessed directly without gRPC.


TableInfo(name='table', sampler_options=uniform: true
, remover_options=fifo: true
is_deterministic: true
, max_size=65000, max_times_sampled=0, rate_limiter_info=samples_per_insert: 1
min_diff: -1.7976931348623157e+308
max_diff: 1.7976931348623157e+308
min_size_to_sample: 1
insert_stats {
  completed: 3
}
sample_stats {
  completed: 3
}
, signature=None, current_size=3, num_episodes=3, num_deleted_episodes=0, num_unique_samples=2, table_worker_time=sleeping_ms: 78794
)

In [6]:
dataset2=reverb.TimestepDataset.from_table_signature("localhost:8000","table",max_in_flight_samples_per_worker=10)

In [8]:
next(iter(dataset2))

[reverb/cc/client.cc:165] Sampler and server are owned by the same process (1496204) so Table table is accessed directly without gRPC.


ReplaySample(info=SampleInfo(key=<tf.Tensor: shape=(), dtype=uint64, numpy=17596500392063279357>, probability=<tf.Tensor: shape=(), dtype=float64, numpy=1.0>, table_size=<tf.Tensor: shape=(), dtype=int64, numpy=4>, priority=<tf.Tensor: shape=(), dtype=float64, numpy=1.0>, times_sampled=<tf.Tensor: shape=(), dtype=int32, numpy=11>), data={'state': <tf.Tensor: shape=(), dtype=float32, numpy=0.49774885>, 'environment': <tf.Tensor: shape=(1, 1, 2), dtype=float32, numpy=array([[[0.8454232 , 0.04760706]]], dtype=float32)>})

In [85]:
replay=dataset2.skip(1)

In [86]:
replay=next(iter(replay))

In [87]:
replay.data

{'state': <tf.Tensor: shape=(), dtype=float32, numpy=0.57800925>,
 'environment': <tf.Tensor: shape=(5, 5, 2), dtype=float32, numpy=
 array([[[0.22750628, 0.5832697 ],
         [0.86505675, 0.10401309],
         [0.20687664, 0.3058145 ],
         [0.2234062 , 0.8591584 ],
         [0.6675446 , 0.27603495]],
 
        [[0.8345144 , 0.45825958],
         [0.74715424, 0.78297615],
         [0.7780688 , 0.41035748],
         [0.03698099, 0.67571926],
         [0.3375945 , 0.43015003]],
 
        [[0.47619116, 0.7298639 ],
         [0.248451  , 0.8680376 ],
         [0.6604856 , 0.86580634],
         [0.69490933, 0.98098385],
         [0.03313601, 0.88633764]],
 
        [[0.7755816 , 0.10151064],
         [0.6948798 , 0.6425749 ],
         [0.09303379, 0.5171981 ],
         [0.6281872 , 0.5559063 ],
         [0.37633646, 0.48895645]],
 
        [[0.04617739, 0.10574234],
         [0.28436494, 0.7769892 ],
         [0.3978827 , 0.54584026],
         [0.702816  , 0.56703997],
         [0.828

In [88]:
table.info

TableInfo(name='table', sampler_options=uniform: true
, remover_options=fifo: true
is_deterministic: true
, max_size=65000, max_times_sampled=0, rate_limiter_info=samples_per_insert: 1
min_diff: -1.7976931348623157e+308
max_diff: 1.7976931348623157e+308
min_size_to_sample: 1
insert_stats {
  completed: 4
}
sample_stats {
  completed: 260
}
, signature={'environment': TensorSpec(shape=(None, None, 2), dtype=tf.float32, name=None), 'state': TensorSpec(shape=(), dtype=tf.float32, name=None)}, current_size=4, num_episodes=4, num_deleted_episodes=0, num_unique_samples=4, table_worker_time=sleeping_ms: 453487
)

In [98]:
import asyncio
# Online Python compiler (interpreter) to run Python online.
# Write Python 3 code in this online editor and run it.
async def A():
    print("HOla")
    await asyncio.sleep(5)
    print("OO")
    
async def B():
    print("COLA")
    await asyncio.sleep(5)
    print("CC")
    
asyncio.gather(A(),B())

<_GatheringFuture pending>

HOla
COLA
OO
CC


In [33]:
a=client.sample("table",2)

In [34]:
names=["environment","state"]
a= [ dict(zip(names,z[0].data)) for z in a]

In [35]:
a

[{'environment': array(0.98913467, dtype=float32),
  'state': array([[[0.18964303, 0.66071284],
          [0.07515097, 0.9688363 ],
          [0.37223458, 0.05682504],
          [0.8417206 , 0.4246173 ],
          [0.05307496, 0.7297653 ]],
  
         [[0.95934165, 0.5798961 ],
          [0.37970984, 0.03373742],
          [0.1492467 , 0.12759602],
          [0.8235059 , 0.6620413 ],
          [0.33073735, 0.96749485]],
  
         [[0.31027055, 0.5251578 ],
          [0.07770598, 0.5633315 ],
          [0.23489213, 0.46311712],
          [0.26119328, 0.14406645],
          [0.5889164 , 0.598019  ]],
  
         [[0.6984408 , 0.91712475],
          [0.18235064, 0.02049029],
          [0.4754548 , 0.35344374],
          [0.9772159 , 0.51109827],
          [0.81730187, 0.6915761 ]],
  
         [[0.6174283 , 0.43172526],
          [0.00889611, 0.05513704],
          [0.35045362, 0.26662838],
          [0.36207092, 0.63193023],
          [0.9768071 , 0.98974967]]], dtype=float32)},
 {'en

In [48]:
table.can_insert(10000000)

True

In [52]:
client.server_info()["table"]

TableInfo(name='table', sampler_options=uniform: true
, remover_options=fifo: true
is_deterministic: true
, max_size=65000, max_times_sampled=0, rate_limiter_info=samples_per_insert: 1
min_diff: -1.7976931348623157e+308
max_diff: 1.7976931348623157e+308
min_size_to_sample: 1
insert_stats {
  completed: 4
}
sample_stats {
  completed: 12
}
, signature={'environment': TensorSpec(shape=(), dtype=tf.float32, name=None), 'state': TensorSpec(shape=(None, None, 2), dtype=tf.float32, name=None)}, current_size=4, num_episodes=1, num_deleted_episodes=0, num_unique_samples=4, table_worker_time=sleeping_ms: 3925900
)

In [56]:
class X:
    @property
    def random(self):
        return random.randint(0,100)

In [55]:
import random


28

In [57]:
a=X()

In [62]:
a.random

51

In [22]:
import reverb
import tensorflow as tf
client=reverb.Client(f"localhost:50051")

In [23]:
client.server_info()["mpg"].current_size

1696

In [189]:
x= next(iter(client.sample("mpg",1)))

In [190]:
x,y,z,w=x[0].data

In [4]:
import reverb

In [3]:
reverb.TimestepDataset.from_table_signature("localhost:50051", "mpg",
                                                                  max_in_flight_samples_per_worker=192,
                                                            get_signature_timeout_secs=None)

2023-06-25 07:49:42.749941: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-06-25 07:49:42.749984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: fedora
2023-06-25 07:49:42.749995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: fedora
2023-06-25 07:49:42.750149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.116.4
2023-06-25 07:49:42.750178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.116.4
2023-06-25 07:49:42.750188: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.116.4


<TimestepDataset element_spec=ReplaySample(info=SampleInfo(key=TensorSpec(shape=(), dtype=tf.uint64, name=None), probability=TensorSpec(shape=(), dtype=tf.float64, name=None), table_size=TensorSpec(shape=(), dtype=tf.int64, name=None), priority=TensorSpec(shape=(), dtype=tf.float64, name=None), times_sampled=TensorSpec(shape=(), dtype=tf.int32, name=None)), data=(TensorSpec(shape=(None, None, 2), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None)))>

In [30]:
import pandas
df=pandas.read_json("actors_stats_fedora.ndjson",lines=True)